In [117]:
import os
from google.cloud import bigquery
import pandas

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "finance-analytics-291816-4e2d4b15e5d5 5.json"

client = bigquery.Client()

#Test
sql="""
SELECT DISTINCT INSTITUTION_NAME FROM `finance-analytics-291816.BI_Sales_Chatbot.layer4_FT_VW_FACT_DAILY_SALES_DATA_ANALYSIS` 
"""
query_job = client.query(
    sql,
    job_id_prefix='test_sql_query'
)
print(sql)
df_v1 = query_job.to_dataframe()
# df.to_csv(r'C://Users//P90020194//Desktop//Projects//Extracted_Tables_2//Customer_Table')

# print(df)
df_v1.head()


SELECT DISTINCT INSTITUTION_NAME FROM `finance-analytics-291816.BI_Sales_Chatbot.layer4_FT_VW_FACT_DAILY_SALES_DATA_ANALYSIS` 



e:\Sarthak\Prod_Deployment\v14-Sreyas - Copy\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,INSTITUTION_NAME
0,Army-MH-Bathinda
1,Ankura Hospitals Khammam
2,MANIPAL HOSPITALS
3,Acharya Vinoba Bhave Hosp-Wardha
4,Tata 1mg Health and Solutions Pvt l


# Step 3: Get Table Schema (Columns, Types, Constraints

In [118]:
import pandas as pd
import json
from typing import Dict, List, Optional, Any
import warnings

In [119]:
# Your specific table details
PROJECT_ID = "finance-analytics-291816"
DATASET_ID = "BI_Sales_Chatbot"
TABLE_ID = "layer4_FT_VW_FACT_DAILY_SALES_DATA_ANALYSIS"
TABLE_ID="Product"
TABLE_ID="layer4_FT_VW_FACT_SECONDARY_SALES_DATA"
TABLE_ID="VW_FT_FACT_EXTERNAL_SALES"
TABLE_ID="CUSTOMER"

In [120]:

def get_table_schema(client: bigquery.Client, dataset_id: str, table_id: str) -> pd.DataFrame:
    """
    Extract table schema including column names, types, and constraints
    """
    try:
        # Get table reference
        table_ref = client.dataset(dataset_id).table(table_id)
        table = client.get_table(table_ref)
        
        schema_data = []
        for field in table.schema:
            schema_data.append({
                'column_name': field.name,
                'data_type': field.field_type,
                'mode': field.mode,  # NULLABLE, REQUIRED, REPEATED
                'is_nullable': field.mode == 'NULLABLE',
                'description': field.description or '',
                'max_length': getattr(field, 'max_length', None)
            })
        
        schema_df = pd.DataFrame(schema_data)
        print(f"✅ Schema extracted for {dataset_id}.{table_id}")
        print(f"📊 Found {len(schema_df)} columns")
        return schema_df
    
    except Exception as e:
        print(f"❌ Error getting table schema: {e}")
        return pd.DataFrame()



print(f"🎯 Target Table: {PROJECT_ID}.{DATASET_ID}.{TABLE_ID}")

# Test the connection and get basic schema
schema_df = get_table_schema(client, DATASET_ID, TABLE_ID)
print("\n📋 Table Schema Preview:")
if not schema_df.empty:
    print(schema_df[['column_name', 'data_type', 'mode']].head(10))
    print(f"\n... and {max(0, len(schema_df) - 10)} more columns")
else:
    print("⚠️  Could not retrieve schema. Check your credentials and table access.")

🎯 Target Table: finance-analytics-291816.BI_Sales_Chatbot.CUSTOMER
✅ Schema extracted for BI_Sales_Chatbot.CUSTOMER
📊 Found 15 columns

📋 Table Schema Preview:
             column_name data_type      mode
0          CUSTOMER_CODE    STRING  NULLABLE
1          CUSTOMER_NAME    STRING  NULLABLE
2           COUNTRY_CODE    STRING  NULLABLE
3           COUNTRY_NAME    STRING  NULLABLE
4             STATE_CODE    STRING  NULLABLE
5             STATE_NAME    STRING  NULLABLE
6             STATE_ZONE    STRING  NULLABLE
7         CUSTOMER_GROUP    STRING  NULLABLE
8  CUSTOMER_BUYING_GROUP    STRING  NULLABLE
9      PUBLIC_OR_PRIVATE    STRING  NULLABLE

... and 5 more columns


In [121]:
schema_df[(schema_df['data_type'] == "FLOAT") | (schema_df['data_type'] == "NUMERIC")]['column_name'].to_list()

[]

# Step 4: Identify Primary Keys (Based on Constraints and Patterns)

In [122]:
def identify_primary_keys(client: bigquery.Client, dataset_id: str, table_id: str, 
                         schema_df: pd.DataFrame) -> List[str]:
    """
    Identify potential primary keys based on:
    1. Column names (contains 'id', 'key', 'pk')
    2. Uniqueness check
    3. Non-null constraint
    """
    potential_pks = []
    
    # Check for typical PK naming patterns
    pk_patterns = ['id', 'key', 'pk', '_id']
    for _, row in schema_df.iterrows():
        col_name = row['column_name'].lower()
        if any(pattern in col_name for pattern in pk_patterns):
            if not row['is_nullable']:  # Should not be nullable
                potential_pks.append(row['column_name'])
    
    # Verify uniqueness for potential PKs
    verified_pks = []
    for pk_col in potential_pks:
        try:
            query = f"""
            SELECT 
                COUNT(*) as total_rows,
                COUNT(DISTINCT {pk_col}) as unique_values
            FROM `{client.project}.{dataset_id}.{table_id}`
            """
            result = client.query(query).result()
            for row in result:
                if row.total_rows == row.unique_values and row.total_rows > 0:
                    verified_pks.append(pk_col)
                    print(f"✅ Verified PK: {pk_col} ({row.total_rows} unique values)")
                else:
                    print(f"⚠️  {pk_col} is not unique ({row.unique_values}/{row.total_rows})")
        except Exception as e:
            print(f"❌ Error checking {pk_col}: {e}")
    
    return verified_pks

# Step 5: Identify Categorical/ Numerical Columns

In [123]:
def identify_categorical_columns(schema_df: pd.DataFrame, 
                               max_distinct_threshold: int = 100) -> List[str]:
    """
    Identify potential categorical columns based on data type
    """
    categorical_cols = []
    
    for _, row in schema_df.iterrows():
        col_name = row['column_name']
        data_type = row['data_type']
        
        # String columns are potential categoricals
        if data_type in ['STRING', 'BYTES']:
            categorical_cols.append(col_name)
        # Integer columns with specific naming patterns might be categorical
        elif data_type in ['INTEGER', 'INT64'] and any(
            pattern in col_name.lower() 
            for pattern in ['type', 'status', 'category', 'grade', 'level', 'rating']
        ):
            categorical_cols.append(col_name)
    
    print(f"📊 Identified {len(categorical_cols)} potential categorical columns")
    return categorical_cols

In [124]:
def identify_numerical_columns(schema_df: pd.DataFrame) -> List[str]:
    """
    Identify numerical columns for statistical analysis
    """
    numerical_cols = []
    
    for _, row in schema_df.iterrows():
        col_name = row['column_name']
        data_type = row['data_type']
        
        # All numeric data types
        if data_type in ['INTEGER', 'INT64', 'FLOAT', 'FLOAT64', 'NUMERIC', 'BIGNUMERIC']:
            numerical_cols.append(col_name)
    
    print(f"📊 Identified {len(numerical_cols)} numerical columns")
    return numerical_cols

# Step 6: Extract Distinct Values for Categorical Columns

In [125]:
def get_categorical_distinct_values(client: bigquery.Client, dataset_id: str, table_id: str,
                                  categorical_cols: List[str], 
                                  max_distinct_per_column: int = 100000) -> Dict[str, List]:
    """
    Extract distinct values for each categorical column
    """
    distinct_values = {}
    
    for col in categorical_cols:
        try:
            query = f"""
            SELECT 
                {col},
                COUNT(*) as frequency
            FROM `{client.project}.{dataset_id}.{table_id}`
            WHERE {col} IS NOT NULL
            GROUP BY {col}
            ORDER BY frequency DESC
            LIMIT {max_distinct_per_column}
            """
            
            result = client.query(query).result()
            values_list = []
            total_frequency = 0
            
            for row in result:
                values_list.append({
                    'value': row[0],
                    'frequency': row[1]
                })
                total_frequency += row[1]
            
            distinct_values[col] = {
                'values': [item['value'] for item in values_list],
                'value_counts': values_list,
                'total_distinct': len(values_list),
                'total_frequency': total_frequency
            }
            
            print(f"✅ {col}: {len(values_list)} distinct values extracted")
            
        except Exception as e:
            print(f"❌ Error extracting values for {col}: {e}")
            distinct_values[col] = {'values': [], 'error': str(e)}
    
    return distinct_values

# Step 6.1- Get Numerial olcumn Statistics

In [126]:
def get_numerical_column_statistics(client: bigquery.Client, dataset_id: str, table_id: str,
                                  numerical_cols: List[str]) -> Dict[str, Dict]:
    """
    Extract statistical summary for numerical columns
    """
    numerical_stats = {}
    
    for col in numerical_cols:
        try:
            query = f"""
            SELECT 
                COUNT(*) as total_rows,
                COUNT({col}) as non_null_rows,
                COUNT(*) - COUNT({col}) as null_rows,
                MIN({col}) as min_value,
                MAX({col}) as max_value,
                AVG({col}) as mean_value,
                STDDEV({col}) as std_dev,
                -- Percentiles for distribution analysis
                APPROX_QUANTILES({col}, 100)[OFFSET(25)] as percentile_25,
                APPROX_QUANTILES({col}, 100)[OFFSET(50)] as median,
                APPROX_QUANTILES({col}, 100)[OFFSET(75)] as percentile_75,
                -- Count distinct values to check if it's actually categorical
                COUNT(DISTINCT {col}) as distinct_count
            FROM `{client.project}.{dataset_id}.{table_id}`
            """
            
            result = client.query(query).result()
            
            for row in result:
                # Check if this "numerical" column is actually categorical (low distinct count)
                is_actually_categorical = row.distinct_count <= 50 and row.non_null_rows > 0
                
                numerical_stats[col] = {
                    'total_rows': row.total_rows,
                    'non_null_rows': row.non_null_rows,
                    'null_rows': row.null_rows,
                    'null_percentage': (row.null_rows / row.total_rows * 100) if row.total_rows > 0 else 0,
                    'distinct_count': row.distinct_count,
                    'min_value': float(row.min_value) if row.min_value is not None else None,
                    'max_value': float(row.max_value) if row.max_value is not None else None,
                    'mean_value': float(row.mean_value) if row.mean_value is not None else None,
                    'std_dev': float(row.std_dev) if row.std_dev is not None else None,
                    'median': float(row.median) if row.median is not None else None,
                    'percentile_25': float(row.percentile_25) if row.percentile_25 is not None else None,
                    'percentile_75': float(row.percentile_75) if row.percentile_75 is not None else None,
                    'is_actually_categorical': is_actually_categorical
                }
                
                # Calculate range and IQR
                if row.min_value is not None and row.max_value is not None:
                    numerical_stats[col]['range'] = float(row.max_value) - float(row.min_value)
                
                if row.percentile_75 is not None and row.percentile_25 is not None:
                    numerical_stats[col]['iqr'] = float(row.percentile_75) - float(row.percentile_25)
                
                mean_display = f"{row.mean_value:.2f}" if row.mean_value is not None else "N/A"
                print(f"✅ {col}: min={row.min_value}, max={row.max_value}, mean={mean_display}")
                
                if is_actually_categorical:
                    print(f"   ⚠️  {col} has only {row.distinct_count} distinct values - consider treating as categorical")
                    
        except Exception as e:
            print(f"❌ Error analyzing {col}: {e}")
            numerical_stats[col] = {'error': str(e)}
    
    return numerical_stats

# Step 7: Create Complete Schema Summary

In [127]:
def create_schema_summary(client: bigquery.Client, dataset_id: str, table_id: str) -> Dict[str, Any]:
    """
    Create a comprehensive schema summary combining all extracted information
    """
    print(f"🔍 Analyzing table: {dataset_id}.{table_id}")
    
    # Get basic schema
    schema_df = get_table_schema(client, dataset_id, table_id)
    if schema_df.empty:
        return {}
    
    # Identify primary keys
    print("\n🔑 Identifying Primary Keys...")
    primary_keys = identify_primary_keys(client, dataset_id, table_id, schema_df)
    
    # Identify categorical columns
    print("\n📊 Identifying Categorical Columns...")
    categorical_cols = identify_categorical_columns(schema_df)
    
    # Get distinct values for categorical columns
    print("\n📋 Extracting Distinct Values...")
    distinct_values = get_categorical_distinct_values(
        client, dataset_id, table_id, categorical_cols
    )
    # Identify numerical columns
    print("\n🔢 Identifying Numerical Columns...")
    numerical_cols = identify_numerical_columns(schema_df)
    
    # Get numerical statistics
    print("\n📈 Analyzing Numerical Columns...")
    numerical_stats = get_numerical_column_statistics(
        client, dataset_id, table_id, numerical_cols
    )
    
    # Create summary
    summary = {
        'table_info': {
            'dataset_id': dataset_id,
            'table_id': table_id,
            'total_columns': len(schema_df)
        },
        'schema': schema_df.to_dict('records'),
        'primary_keys': primary_keys,
        'categorical_columns': categorical_cols,
        'numerical_columns': numerical_cols,  # ADD THIS
        'distinct_values': distinct_values,
        'numerical_statistics': numerical_stats,  # ADD THIS
        'column_types': {
            'string_columns': schema_df[schema_df['data_type'].isin(['STRING', 'BYTES'])]['column_name'].tolist(),
            'numeric_columns': schema_df[schema_df['data_type'].isin(['INTEGER', 'INT64', 'FLOAT', 'FLOAT64', 'NUMERIC'])]['column_name'].tolist(),
            'datetime_columns': schema_df[schema_df['data_type'].isin(['TIMESTAMP', 'DATETIME', 'DATE', 'TIME'])]['column_name'].tolist(),
            'boolean_columns': schema_df[schema_df['data_type'] == 'BOOLEAN']['column_name'].tolist()
        }
    }
    
    return summary

# Step 8: Run Complete Analysis for Your Table


## Display Summary

In [128]:
def display_summary(summary: Dict[str, Any]):
    """
    Display the schema summary in a readable format
    """
    if not summary:
        print("❌ No summary to display")
        return
    
    print("=" * 60)
    print("📊 SCHEMA ANALYSIS SUMMARY")
    print("=" * 60)
    
    # Table info
    table_info = summary['table_info']
    print(f"\n🏷️  Table: {table_info['dataset_id']}.{table_info['table_id']}")
    print(f"📊 Total Columns: {table_info['total_columns']}")
    
    # Primary keys
    print(f"\n🔑 Primary Keys: {summary['primary_keys']}")
    
    # Column types breakdown
    col_types = summary['column_types']
    print(f"\n📈 Column Type Breakdown:")
    for col_type, cols in col_types.items():
        if cols:
            print(f"   {col_type}: {len(cols)} columns")
    
    # Categorical columns with distinct values
    print(f"\n📋 Categorical Columns ({len(summary['categorical_columns'])}):")
    for col in summary['categorical_columns']:
        if col in summary['distinct_values']:
            distinct_info = summary['distinct_values'][col]
            if 'values' in distinct_info:
                print(f"   {col}: {distinct_info['total_distinct']} distinct values")
                # Show top 5 values
                if distinct_info['value_counts']:
                    top_values = distinct_info['value_counts'][:5]
                    print(f"      Top values: {[item['value'] for item in top_values]}")

    # Numerical columns summary
    if 'numerical_statistics' in summary and summary['numerical_statistics']:
        print(f"\n🔢 Numerical Columns ({len(summary['numerical_columns'])}):")
        for col, stats in summary['numerical_statistics'].items():
            if 'error' not in stats:
                print(f"   {col}:")
                print(f"      Range: {stats.get('min_value', 'N/A')} to {stats.get('max_value', 'N/A')}")
                print(f"      Mean: {stats.get('mean_value', 'N/A')}")
                print(f"      Distinct Values: {stats.get('distinct_count', 'N/A')}")
                if stats.get('is_actually_categorical'):
                    print(f"      ⚠️  Low distinct count - consider as categorical")

In [129]:
print("🚀 Starting Complete Schema Analysis for your Sales Data table...")
schema_summary = create_schema_summary(client, DATASET_ID, TABLE_ID)

# Display the results
if schema_summary:
    display_summary(schema_summary)
else:
    print("❌ Failed to analyze schema. Please check your BigQuery credentials and table access.")

🚀 Starting Complete Schema Analysis for your Sales Data table...
🔍 Analyzing table: BI_Sales_Chatbot.CUSTOMER
✅ Schema extracted for BI_Sales_Chatbot.CUSTOMER
📊 Found 15 columns

🔑 Identifying Primary Keys...

📊 Identifying Categorical Columns...
📊 Identified 13 potential categorical columns

📋 Extracting Distinct Values...
✅ CUSTOMER_CODE: 100000 distinct values extracted
✅ CUSTOMER_NAME: 100000 distinct values extracted
✅ COUNTRY_CODE: 150 distinct values extracted
✅ COUNTRY_NAME: 150 distinct values extracted
✅ STATE_CODE: 409 distinct values extracted
✅ STATE_NAME: 40 distinct values extracted
✅ STATE_ZONE: 6 distinct values extracted
✅ CUSTOMER_GROUP: 82500 distinct values extracted
✅ CUSTOMER_BUYING_GROUP: 29654 distinct values extracted
✅ PUBLIC_OR_PRIVATE: 1 distinct values extracted
✅ CREATED_BY: 1 distinct values extracted
✅ UPDATED_BY: 1 distinct values extracted
✅ SEC_SALES_DIST_ID: 37 distinct values extracted

🔢 Identifying Numerical Columns...
📊 Identified 0 numerical co

# Step9- Saving the schema

In [130]:
def export_schema_for_synthetic_data(summary: Dict[str, Any], output_file: str = "schema_for_synthetic_data.json"):
    """
    Export schema in a format optimized for synthetic data generation
    """
    if not summary:
        print("❌ No summary to export")
        return
    
    # Create synthetic data friendly format
    synthetic_schema = {
        'table_name': f"{summary['table_info']['dataset_id']}.{summary['table_info']['table_id']}",
        'columns': {}
    }
    
    for col_info in summary['schema']:
        col_name = col_info['column_name']
        
        # Base column info
        column_config = {
            'data_type': col_info['data_type'],
            'nullable': col_info['is_nullable'],
            'is_primary_key': col_name in summary['primary_keys']
        }
        
        # Add categorical values if available
        if col_name in summary['distinct_values']:
            distinct_info = summary['distinct_values'][col_name]
            if 'values' in distinct_info:
                column_config['categorical_values'] = distinct_info['values']
                column_config['value_distribution'] = {
                    item['value']: item['frequency'] 
                    for item in distinct_info['value_counts']
                }
        
        synthetic_schema['columns'][col_name] = column_config

        # Add numerical statistics if available
        if col_name in summary.get('numerical_statistics', {}):
            num_stats = summary['numerical_statistics'][col_name]
            if 'error' not in num_stats:
                column_config['numerical_stats'] = {
                    'min_value': num_stats.get('min_value'),
                    'max_value': num_stats.get('max_value'),
                    'mean_value': num_stats.get('mean_value'),
                    'std_dev': num_stats.get('std_dev'),
                    'median': num_stats.get('median'),
                    'distinct_count': num_stats.get('distinct_count'),
                    'null_percentage': num_stats.get('null_percentage'),
                    'is_actually_categorical': num_stats.get('is_actually_categorical', False)
                }
    
    # Save to file
    with open(output_file, 'w') as f:
        json.dump(synthetic_schema, f, indent=2, default=str)
    
    print(f"✅ Schema exported to {output_file}")
    return synthetic_schema

In [131]:
# Step 10: Export Schema for Synthetic Data Generation
print("\n📤 Exporting schema for synthetic data generation...")
synthetic_schema = export_schema_for_synthetic_data(
    schema_summary, 
    output_file=f"sales_data_schema_for_synthetic_{TABLE_ID}.json"
)

# Show a preview of what was exported
if synthetic_schema:
    print(f"\n📊 Exported Schema Preview:")
    print(f"Table: {synthetic_schema['table_name']}")
    print(f"Total Columns: {len(synthetic_schema['columns'])}")
    
    # Show first few columns as example
    sample_columns = list(synthetic_schema['columns'].items())[:3]
    for col_name, col_config in sample_columns:
        print(f"\n📋 {col_name}:")
        print(f"   Type: {col_config['data_type']}")
        print(f"   Nullable: {col_config['nullable']}")
        print(f"   Primary Key: {col_config['is_primary_key']}")
        if 'categorical_values' in col_config:
            values_preview = col_config['categorical_values'][:5]
            print(f"   Sample Values: {values_preview}")
            if len(col_config['categorical_values']) > 5:
                print(f"   ... and {len(col_config['categorical_values']) - 5} more values")

print("\n🎉 Schema extraction pipeline configured for your Sales Data table!")
print("📝 Ready to run - your table details are already set:")
print(f"   📊 Project: {PROJECT_ID}")  
print(f"   📂 Dataset: {DATASET_ID}")
print(f"   📋 Table: {TABLE_ID}")
print("\n▶️  Now you can run each step to extract and validate your schema!")
print("🚀 The analysis will automatically run for your specific table.")


📤 Exporting schema for synthetic data generation...
✅ Schema exported to sales_data_schema_for_synthetic_CUSTOMER.json

📊 Exported Schema Preview:
Table: BI_Sales_Chatbot.CUSTOMER
Total Columns: 15

📋 CUSTOMER_CODE:
   Type: STRING
   Nullable: True
   Primary Key: False
   Sample Values: ['889629', '509900', '908093', '891553', '895908']
   ... and 99995 more values

📋 CUSTOMER_NAME:
   Type: STRING
   Nullable: True
   Primary Key: False
   Sample Values: ['WALGREEN CO.', 'WALGREEN CO', 'WALGREEN EASTERN CO., INC.', "ALBERTSON'S PHARMACY", 'THRIFTY PAYLESS']
   ... and 99995 more values

📋 COUNTRY_CODE:
   Type: STRING
   Nullable: True
   Primary Key: False
   Sample Values: ['IN', 'US', 'DE', 'IT', 'FR']
   ... and 145 more values

🎉 Schema extraction pipeline configured for your Sales Data table!
📝 Ready to run - your table details are already set:
   📊 Project: finance-analytics-291816
   📂 Dataset: BI_Sales_Chatbot
   📋 Table: CUSTOMER

▶️  Now you can run each step to extract 